In [ ]:
import pandas as pd
import requests
import os
from io import StringIO
from datetime import datetime
import re

# Set the working directory where you want to save the data
os.chdir(r"C:\Users\ryanb\OneDrive\Desktop\School\Projects\fantasy point predictor\data")

# Define constants
POSITIONS = ['qb', 'rb', 'wr', 'te', 'k', 'dst']
MIN_YEAR = 2021
CURRENT_YEAR = datetime.now().year
MIN_WEEK = 1
MAX_WEEK = 18

# Dictionary to map full team names (used in projections for DST) to their abbreviations
TEAM_ABBREVIATIONS = {
    "Arizona Cardinals": "ARI", "Atlanta Falcons": "ATL", "Baltimore Ravens": "BAL",
    "Buffalo Bills": "BUF", "Carolina Panthers": "CAR", "Chicago Bears": "CHI",
    "Cincinnati Bengals": "CIN", "Cleveland Browns": "CLE", "Dallas Cowboys": "DAL",
    "Denver Broncos": "DEN", "Detroit Lions": "DET", "Green Bay Packers": "GB",
    "Houston Texans": "HOU", "Indianapolis Colts": "IND", "Jacksonville Jaguars": "JAX",
    "Kansas City Chiefs": "KC", "Las Vegas Raiders": "LV", "Los Angeles Chargers": "LAC",
    "Los Angeles Rams": "LAR", "Miami Dolphins": "MIA", "Minnesota Vikings": "MIN",
    "New England Patriots": "NE", "New Orleans Saints": "NO", "New York Giants": "NYG",
    "New York Jets": "NYJ", "Philadelphia Eagles": "PHI", "Pittsburgh Steelers": "PIT",
    "San Francisco 49ers": "SF", "Seattle Seahawks": "SEA", "Tampa Bay Buccaneers": "TB",
    "Tennessee Titans": "TEN", "Washington Commanders": "WAS",
}

# Dictionary to hold the final, merged data for each position
merged_position_dfs = {pos: pd.DataFrame() for pos in POSITIONS}

# Loop through years and weeks
for year in range(MIN_YEAR, CURRENT_YEAR):
    print(f"Processing Year: {year}")
    for week in range(MIN_WEEK, MAX_WEEK + 1):
        projections_week = {}
        actuals_week = {}

        # Loop through each data type and position
        for data_type in ['projections', 'stats']:
            for position in POSITIONS:
                # Construct URL based on data type and position
                if data_type == 'projections':
                    scoring = "&scoring=PPR" if position in ['rb', 'wr', 'te'] else ""
                    url = f"https://www.fantasypros.com/nfl/projections/{position}.php?week={week}&year={year}{scoring}"
                else:  # 'stats'
                    url = f"https://www.fantasypros.com/nfl/stats/{position}.php?year={year}&week={week}&scoring=PPR&range=week"

                # Fetch and Process Data
                try:
                    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
                    if not response.ok:
                        print(f"Skipping {data_type} for {position.upper()} Week {week}, {year} (URL not found: {response.status_code})")
                        continue
                    tables = pd.read_html(StringIO(response.text))
                    if not tables:
                        print(f"No {data_type} table found for {position.upper()} Week {week}, {year}")
                        continue
                    df = tables[0].copy()

                    # Clean DataFrame based on data type
                    if isinstance(df.columns, pd.MultiIndex):
                        df.columns = [f"{col[0]}_{col[1]}" for col in df.columns]
                        df.rename(columns={'MISC_FPTS': 'FPTS'}, inplace=True)
                    
                    player_col_index = 0 if data_type == 'projections' else 1
                    if len(df.columns) > player_col_index:
                        df.rename(columns={df.columns[player_col_index]: 'Player'}, inplace=True)
                    else:
                        print(f"Could not find player column in {data_type} for {position.upper()} Week {week}, {year}")
                        continue

                    if data_type == 'projections':
                        if position == 'dst':
                            df['Team'] = df['Player'].map(TEAM_ABBREVIATIONS)
                        else:
                            df[['Player', 'Team']] = df['Player'].str.rsplit(' ', n=1, expand=True)
                    else:  # 'stats'
                        df[['Player', 'Team']] = df['Player'].apply(
                            lambda s: pd.Series(
                                re.match(r'^(.*?)\s+\((\w{2,3})\)$', s).groups()
                                if re.match(r'^(.*?)\s+\((\w{2,3})\)$', s)
                                else (s, None)
                            )
                        )

                    # Add Week, Year, Position columns
                    df['Week'] = week
                    df['Year'] = year
                    df['Position'] = position.upper()
                    
                    # Store in the appropriate dictionary
                    if data_type == 'projections':
                        projections_week[position] = df
                    else:
                        actuals_week[position] = df

                except Exception as e:
                    print(f"An error occurred fetching {data_type} for {position.upper()} Week {week}, {year}: {e}")
                    continue

        # Merge the data for the current week and year
        for position in POSITIONS:
            proj_df = projections_week.get(position)
            actual_df = actuals_week.get(position)

            if proj_df is not None and not proj_df.empty:
                if actual_df is not None and not actual_df.empty:
                    merged_df = pd.merge(
                        proj_df,
                        actual_df,
                        on=['Player', 'Week', 'Year'],
                        how='left',
                        suffixes=('_proj', '_actual')
                    )
                else:
                    merged_df = proj_df
                    print(f"No actuals data found for {position.upper()} Week {week}, {year}. Using projections only for this week.")
                
                merged_position_dfs[position] = pd.concat([merged_position_dfs[position], merged_df], ignore_index=True)
            elif proj_df is None or proj_df.empty:
                print(f"No projections data for {position.upper()} Week {week}, {year}. Skipping this position for this week.")

# Save the final merged data to CSV files for each position and store position data into a full dataframe
print("\n--- Saving the final merged data to CSV files ---")
full_merged_df = pd.DataFrame()
for position, df in merged_position_dfs.items():
    if not df.empty:
        filename = f"{position}_merged_data.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {filename}")
        full_merged_df = pd.concat([full_merged_df, df], ignore_index=True)

# Save the full merged dataframe
if not full_merged_df.empty:
    full_filename = "full_merged_data.csv"
    full_merged_df.to_csv(full_filename, index=False)
    print(f"Saved combined file: {full_filename}")

Processing Year: 2021
Processing Year: 2022
